In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove840b300dtxt/glove.840B.300d.txt
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test_labels.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv
/kaggle/input/jigsaw-trai

In [9]:
!pip install transformers


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [10]:
!pip install sentencepiece


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [11]:
MAX_LEN = 192 
DROPOUT = 0.5 # use aggressive dropout
BATCH_SIZE = 16 # per TPU core
TOTAL_STEPS_STAGE1 = 2000
VALIDATE_EVERY_STAGE1 = 200
TOTAL_STEPS_STAGE2 = 200
VALIDATE_EVERY_STAGE2 = 10

### Different learning rate for transformer and head ###
LR_TRANSFORMER = 5e-6
LR_HEAD = 1e-3

PRETRAINED_TOKENIZER=  'jplu/tf-xlm-roberta-large'
PRETRAINED_MODEL = '/kaggle/input/jigsaw-mlm-finetuned-xlm-r-large'
D = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'
D_TRANS = '/kaggle/input/jigsaw-train-multilingual-coments-google-api/'


import os
import numpy as np
import pandas as pd
import sentencepiece
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import transformers
from transformers import TFRobertaModel, AutoTokenizer
import logging
# no extensive logging 
logging.getLogger().setLevel(logging.NOTSET)

AUTO = tf.data.experimental.AUTOTUNE

2.12.0


In [12]:
from tqdm import tqdm

In [13]:
def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)


Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/

In [32]:
# train =  pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
train = pd.read_csv('/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-ru-cleaned.csv')
train['toxic'].sum()
# train.shape

21362

In [28]:
# df.query('toxic==0').sample(sum(df.toxic))
# Get the number of toxic rows (toxic == 1)
num_toxic = train['toxic'].sum()

# Sample an equal number of non-toxic rows (toxic == 0)
non_toxic_sample = train[train['toxic'] == 0].sample(num_toxic)
non_toxic_sample.shape

(21384, 8)

In [14]:
def load_jigsaw_trans(langs=['tr','it','es','ru','fr','pt'], 
                      columns=['comment_text', 'toxic']):
    train_6langs=[]
    for i in range(len(langs)):
        fn = D_TRANS+'jigsaw-toxic-comment-train-google-%s-cleaned.csv'%langs[i]
        train_6langs.append(downsample(pd.read_csv(fn)[columns]))

    return train_6langs

def downsample(df):
    """Subsample the train dataframe to 50%-50%"""
    ds_df= pd.concat([
        df.query('toxic==1'),
        df.query('toxic==0').sample(sum(df.toxic))
    ])
    
    return ds_df
    

train_df = pd.concat(load_jigsaw_trans()) 
val_df = pd.read_csv(D+'validation.csv')
test_df = pd.read_csv(D+'test.csv')
sub_df = pd.read_csv(D+'sample_submission.csv')

In [24]:
train_df.shape

(256496, 2)

In [33]:
train_df.sample(10)

,comment_text,toxic
40468,24/7\n\nArrêtez d'apporter des modifications i...,1
46720,"""'mejorar la suerte de los pobres, pero afianz...",0
46202,Heil Hitler! Heil Hitler! Heil Hitler! Heil Hi...,1
56165,"""::: Cher Samboy, je ne crains pas du tout les...",0
144634,"J'ai ajouté l'invasion de l'Irak en 2003, car ...",0
57729,Me gustaría trasladar a los protestantes del U...,0
88100,Şimdi kadeh kaldırıyoruz\n\nha ha öylesine tos...,1
13301,Красная ссылка\n\nВы недавно заново создали сс...,0
67258,¡Maldito infierno!\nEres tan idiota. ¡Deja de ...,1
29805,Muy buen contribuyente? Mierda. La camarilla n...,1


In [15]:
%%time

def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])
    

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_TOKENIZER)
X_train = regular_encode(train_df.comment_text.values.tolist(), tokenizer, maxlen=MAX_LEN)
X_val = regular_encode(val_df.comment_text.values.tolist(), tokenizer, maxlen=MAX_LEN)
X_test = regular_encode(test_df.content.values.tolist(), tokenizer, maxlen=MAX_LEN)

y_train = train_df.toxic.values.reshape(-1,1)
y_val = val_df.toxic.values.reshape(-1,1)


/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad t

CPU times: user 8min 36s, sys: 23.9 s, total: 9min
Wall time: 41.6 s


In [16]:
def create_dist_dataset(X, y=None, training=False):
    dataset = tf.data.Dataset.from_tensor_slices(X)

    ### Add y if present ###
    if y is not None:
        dataset_y = tf.data.Dataset.from_tensor_slices(y)
        dataset = tf.data.Dataset.zip((dataset, dataset_y))
        
    ### Repeat if training ###
    if training:
        dataset = dataset.shuffle(len(X)).repeat()

    dataset = dataset.batch(global_batch_size).prefetch(AUTO)

    ### make it distributed  ###
    dist_dataset = strategy.experimental_distribute_dataset(dataset)

    return dist_dataset
    

train_dist_dataset = create_dist_dataset(X_train, y_train, True)
val_dist_dataset   = create_dist_dataset(X_val)
test_dist_dataset  = create_dist_dataset(X_test)

In [20]:
X_train.shape

(256496, 192)

In [ ]:
%%time
def create_model_and_optimizer():
    with strategy.scope():
        transformer_layer = TFRobertaModel.from_pretrained(PRETRAINED_MODEL)  
#         transformer_layer = TransformerLayer(base_transformer)
        model = build_model(transformer_layer)
        optimizer_transformer = Adam(learning_rate=LR_TRANSFORMER)
        optimizer_head = Adam(learning_rate=LR_HEAD)
    return model, optimizer_transformer, optimizer_head


def build_model(transformer):
    inp = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
    # Huggingface transformers have multiple outputs, embeddings are the first one
    # let's slice out the first position, the paper says its not worse than pooling
    
    x = transformer(inp)[0][:, 0, :]  
    x = Dropout(DROPOUT)(x)
    ### note, adding the name to later identify these weights for different LR
    out = Dense(1, activation='sigmoid', name='custom_head')(x)
    model = Model(inputs=[inp], outputs=[out])
    
    return model


model, optimizer_transformer, optimizer_head = create_model_and_optimizer()

model.summary()

In [ ]:
train.sample(100)

In [ ]:
validation['lang'].unique()

In [ ]:
train['toxic'].sum()

In [ ]:
(train['toxic'] == 0).sum()

In [ ]:
import numpy as np
import pandas as pd

# Example DataFrame
# test = pd.read_csv('/path/to/your/test.csv')

# Calculate total number of entries
total_entries = len(validation)

# Iterate over each unique language and calculate its percentage
for lang in validation['lang'].unique():
    lang_count = np.sum(validation['lang'] == lang)
    lang_percentage = (lang_count / total_entries) * 100
    print(f"{lang} : {lang_percentage:.2f}%")

In [ ]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
import os

# Function to translate text
def translate_text(text, model_name):
    try:
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)

        # Tokenize and translate the text
        translated = tokenizer([text], return_tensors="pt", padding=True, truncation=True, max_length=2400)
        translated_tokens = model.generate(**translated)
        translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
        return translated_text
    except Exception as e:
        print(f"Error translating text: {text}. Exception: {e}")
        return None  # Return None if an error occurs

# Load dataset
# df = pd.read_csv('/content/sample_data/jigsaw-toxic-comment-train.csv')
df = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')

# Extract comments and labels
comments_to_translate = df['comment_text'].tolist()
labels = df['toxic'].tolist()

# Define MarianMT models for the target languages
models = {
    "es": "Helsinki-NLP/opus-mt-en-es",   # English to Spanish
    "tr": "Helsinki-NLP/opus-mt-en-trk",  # English to Turkish (corrected)
    "ru": "Helsinki-NLP/opus-mt-en-ru",   # English to Russian
    "it": "Helsinki-NLP/opus-mt-en-it",   # English to Italian
    "fr": "Helsinki-NLP/opus-mt-en-fr"    # English to French
}

# Set batch size
batch_size = 100  # Modify this based on available resources
start_idx = 0

# Check if the output file already exists and resume if needed
output_file = 'translated_dataset.csv'
if os.path.exists(output_file):
    translated_df = pd.read_csv(output_file)
    start_idx = len(translated_df) // len(models)  # Calculate start index based on languages translated
else:
    translated_df = pd.DataFrame(columns=['comment', 'toxic', 'language'])

# Process the dataset in batches
for start in range(start_idx, len(comments_to_translate), batch_size):
    end = min(start + batch_size, len(comments_to_translate))
    translated_data = []

    for i, text in enumerate(comments_to_translate[start:end]):
        label = labels[start + i]

        # Translate to each language
        for lang, model_name in models.items():
            print(f"Translating comment {start + i + 1} to {lang}...")
            translation = translate_text(text, model_name)
            if translation:
                translated_data.append({'comment': translation, 'toxic': label, 'language': lang})

    # Convert batch to DataFrame and append to the existing DataFrame
    batch_df = pd.DataFrame(translated_data)
    translated_df = pd.concat([translated_df, batch_df], ignore_index=True)

    # Save the translated dataset after each batch
    translated_df.to_csv(output_file, index=False)
    print(f"Batch {start // batch_size + 1} saved to {output_file}")

print("All translations completed and saved.")


In [ ]:
train.shape

In [ ]:
print(f"Train Shape is  {train.shape[0]}")
print(f"Validation Shape is  {validation.shape[0]}")
print(f"Test Shape is  {test.shape[0]}")

In [ ]:
train.head()

In [ ]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis = 1, inplace = True)

In [ ]:
#check max len of comment_text column to use this for padding in future
pad_len = train['comment_text'].apply(lambda x:len(str(x).split())).max()
print('max len of comment_text column',pad_len)

# Data Preperation

In [ ]:
texts = train.comment_text.values
labels = train.toxic.values

# Step 1: Split into training and temporary sets
x_train, x_temp, y_train, y_temp = train_test_split(
    texts, labels, stratify=labels, test_size=0.3, random_state=42, shuffle=True
)

# Step 2: Split the temporary set into validation and test sets
x_valid, x_test, y_valid, y_test = train_test_split(
    x_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42, shuffle=True
)

# Check the sizes of each set
print(f"Training set size: {len(x_train)}")
print(f"Validation set size: {len(x_valid)}")
print(f"Test set size: {len(x_test)}")

In [ ]:
len(xtrain),len(xvalid)

# Tokenisation and Padding with max len of words in curpus

In [ ]:
test.head()

In [ ]:
#using keras tokenizer  --> word-level tokenizer




# The Tokenizer from Keras is a tool used to preprocess text data 
# for machine learning models. It converts text into sequences of integers
# where each integer represents a unique word in the text corpus.
# Here's a step-by-step explanation of how it works in your code

token = text.Tokenizer(num_words = None)
max_len = 2400 # bec biggest comment in training is 2321
# xtest = test.content.values
# fit_on_texts() builds the word index from the provided texts.
# It scans through all the text samples (xtrain, xvalid, and xtest),
# creating a vocabulary dictionary where each word is mapped to a unique integer.
# This mapping is used later to convert the text into sequences of integers.


token.fit_on_texts(list(x_train) + list(x_valid) + list(x_test))

x_train_seq = token.texts_to_sequences(x_train)
x_valid_seq = token.texts_to_sequences(x_valid)
x_test_seq = token.texts_to_sequences(x_test)
# texts_to_sequences() converts each text sample into a sequence of integers 
# based on the word index created during fit_on_texts(). Each word in the text is
# replaced by its corresponding integer from the word index. 
# If a word is not found in the word index (e.g., it wasn't in the training data), it is ignored.


#zero pad the sequences
x_train_pad = pad_sequences(x_train_seq,maxlen = max_len)
x_valid_pad = pad_sequences(x_valid_seq,maxlen = max_len)
x_test_pad = pad_sequences(x_test_seq,maxlen = max_len)
# pad_sequences() ensures that all sequences have the same length (max_len). 
# Sequences shorter than max_len are padded with zeros at the beginning,
# while sequences longer than max_len are truncated. This step is crucial 
# for ensuring that the input data has a consistent shape when fed into a neural network.
word_index = token.word_index
# word_index is a dictionary where keys are words and values are their corresponding integer indices. 
# This index helps in understanding the mapping between words and integers used in the sequences.

# 1: classification on basic RNN Network

In [ ]:
len(word_index) + 1

In [ ]:
%%time
with strategy.scope():
    
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Input
from tensorflow.keras.models import Model

# Define the RNN cell manually
class CustomRNNCell(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(CustomRNNCell, self).__init__(**kwargs)
#         ensures that the parent class (Layer) is properly initialized,
#         allowing the cell to function as a layer within a model.
        self.units = units

    def build(self, input_shape):
#         The build method is called automatically when the layer is first used 
        self.W = self.add_weight(shape=(input_shape[-1], self.units), initializer='glorot_uniform', name='W')
        
        self.U = self.add_weight(shape=(self.units, self.units), initializer='orthogonal', name='U')
        self.b = self.add_weight(shape=(self.units,), initializer='zeros', name='b')
        # Print the shapes of the weights
        print(f"W shape: {self.W.shape}")
        print(f"U shape: {self.U.shape}")
        print(f"b shape: {self.b.shape}")


    def call(self, inputs, states):
#         The call method defines the forward pass of the RNN cell. It takes two inputs:

        prev_state = states[0]
        print(f" prev state dims {prev_state.shape}")
        h = tf.tanh(tf.matmul(inputs, self.W) + tf.matmul(prev_state, self.U) + self.b)
        return h, [h]

# Build the custom RNN layer
class CustomRNN(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(CustomRNN, self).__init__(**kwargs)
        self.units = units
        self.cell = CustomRNNCell(units)

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        time_steps = tf.shape(inputs)[1]

        # Initialize state and TensorArray for outputs
        state = [tf.zeros((batch_size, self.units))]
        outputs = tf.TensorArray(tf.float32, size=time_steps)

        # Loop through time steps
        for t in tf.range(time_steps):
            x = inputs[:, t, :]
            output, state = self.cell(x, state)
            outputs = outputs.write(t, output)

        # Stack outputs and transpose to match the expected shape
        outputs = outputs.stack()
        outputs = tf.transpose(outputs, [1, 0, 2])  # (batch_size, time_steps, units)
        return outputs

# Define the model
vocab_size = len(word_index) + 1
embedding_dim = 300
max_len = 2400  # Example length
rnn_units = 100

# x --> emb  --> 2400*300  X  W (300 * 100) -->  2400*100 + ( (prev_state*U) -->   2400*100      * 100*100 
#

inputs = Input(shape=(max_len,))
# pdb.set_trace()
with strategy.scope():
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)(inputs)
    print(x.shape)
    x = CustomRNN(rnn_units)(x)
    x = tf.reduce_mean(x, axis=1)  # Aggregating the outputs (e.g., via mean or sum)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


In [ ]:
#using strategy to run the TPU
model.fit(x_train_pad,y_train,epochs = 5,batch_size = 64*strategy.num_replicas_in_sync)

In [ ]:
predictions = model.predict(x_test_pad)


In [ ]:
# Assuming predictions are probabilities, you might want to convert them to binary class labels
# For binary classification, you can use a threshold of 0.5 to get class labels
predicted_labels = (predictions > 0.5).astype(int)

# Create a DataFrame
df_predictions = pd.DataFrame(predicted_labels, columns=['Predicted_Label'])

# Save to CSV
df_predictions.to_csv('predictions.csv', index=False)

In [ ]:
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import numpy as np

In [ ]:
pred_val = model.predict(x_test_pad)

In [ ]:
model_score = roc_auc_score(y_test,pred_val)

In [ ]:
model_score

----------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------

# 2 : classification with pretrained glove word Embedding with Basic LSTM Model

In [ ]:
# load glove vector in a dictionary

embeddings_index = {}
f = open('/kaggle/input/glove840b300dtxt/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(value) for value in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found {} word vectors'.format(len(embeddings_index)))

In [ ]:
#create an embedding metrics for the words which are part of our datasets
embedding_metrics = np.zeros((len(word_index) + 1,300))
for word,i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_metrics[i] = embedding_vector
    

In [ ]:
embedding_metrics.shape

In [ ]:
%%time
with strategy.scope():
    
    #simple LSTM Model
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
             300, weights = [embedding_metrics],input_length = max_len,trainable = False))
    model.add(LSTM(100,dropout = 0.3, recurrent_dropout = 0.3))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model.summary()

In [ ]:
model.fit(x_train_pad,ytrain,epochs = 5, batch_size = 64*strategy.num_replicas_in_sync)

In [ ]:
lstm_pred = model.predict(x_valid_pad)

In [ ]:
model_accuracy = roc_auc_score(yvalid,lstm_pred)
# model_accuracy_ls.append({'model':'LSTM','AUC_SCORE':model_accuracy})
mo

In [ ]:
model_accuracy_ls

# Classification on GRU(gated Recurrent Unit) Netwrok

In [ ]:
%%time

with strategy.scope():
    
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                       300,
                       weights = [embedding_metrics],
                       input_length = max_len,
                       trainable = False))
    model.add(SpatialDropout1D(0.3))
    model.add(GRU(300))
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

In [ ]:
model.fit(x_train_pad,ytrain, epochs = 5,batch_size = 64*strategy.num_replicas_in_sync)

In [ ]:
gru_pred = model.predict(x_valid_pad)

In [ ]:
model_accuracy = roc_auc_score(yvalid,gru_pred)
model_accuracy_ls.append({'model':'GRU','AUC_SCORE':model_accuracy})

In [ ]:
model_accuracy_ls

# Classification with BiDirectional RNN's

In [ ]:
%%time

with strategy.scope():
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                       300,
                       weights = [embedding_metrics],
                       input_length = max_len,
                       trainable = False))
    model.add(Bidirectional(LSTM(300, dropout = 0.3, recurrent_dropout = 0.3)))
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
model.summary()

In [ ]:
model.fit(x_train_pad, ytrain, epochs = 5, batch_size = 64* strategy.num_replicas_in_sync)

In [ ]:
bidirectional_score = model.predict(x_valid_pad)

In [ ]:
model_accuracy = roc_auc_score(yvalid,bidirectional_score)
model_accuracy_ls.append({'model':'Bidircetional_RNN','AUC_SCORE':model_accuracy})

In [ ]:
#visualization of results obtained from various Deep learning Model
results_df = pd.DataFrame(model_accuracy_ls).sort_values(by = 'AUC_SCORE', ascending = False)
results_df.style.background_gradient(cmap = 'Blues')

# BERT Model

In [ ]:
!pip install transformers

In [ ]:
import transformers
from tokenizers import BertWordPieceTokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    """
    Encoder for encoding the text into sequence of integers for BERT Input
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

# Tokenization

In [ ]:
# load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

#save the loaded tokenizer locally
tokenizer.save_pretrained('.')

#reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt',lowercase = False)
fast_tokenizer

In [ ]:
train.head()

In [ ]:
x_train = fast_encode(train.comment_text.astype(str), fast_tokenizer, maxlen=192)
x_valid = fast_encode(validation.comment_text.astype(str), fast_tokenizer, maxlen=192)
x_test = fast_encode(test.content.astype(str), fast_tokenizer, maxlen=192)

y_train = train.toxic.values
y_valid = validation.toxic.values

In [ ]:
BATCH_SIZE = 512
AUTO = tf.data.experimental.AUTOTUNE

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [ ]:
def build_model(transformer, max_len=512):
    """
    funtion for training Bert Model
    """
    
    input_word_ids = Input(shape = (max_len,), dtype = tf.int32, name = "input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:,0,:]
    out = Dense(1, activation = 'sigmoid')(cls_token)
    model = Model(inputs = input_word_ids, outputs = out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model
    
    

In [ ]:
# start model training 
with strategy.scope():
    transformer_layer = (
    transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer,max_len = 192)
model.summary()

In [ ]:
n_steps = x_train.shape[0] # BATCH_SIZE
train_history = model.fit(train_dataset,
                         steps_per_epoch = n_steps,
                         validation_data=valid_dataset,
                         epochs=2
                         )

In [ ]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_1 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=2*2
)

In [ ]:
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [ ]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)